In [70]:

import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [37]:
# read csv
path='../data/heb_scrape.csv'
heb_df=pd.read_csv(path)
heb_df

,brand,category,coupon,features,id,image,name,price,size,uomSalePrice
0,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931316,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics Whole Milk, 1 gal",5.85,", 1 gal",($0.05/oz)
1,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931317,https://images.heb.com/is/image/HEBGrocery/prd...,Central Market Organics Reduced Fat 2% Milkfat...,5.85,"fat Milk, 1 gal",($0.05/oz)
2,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS. Primo Pick.,671076,https://images.heb.com/is/image/HEBGrocery/prd...,H‑E‑B Select Ingredients MooTopia Lactose Free...,4.10,", 1/2 gal",($0.07/oz)
3,H-E-B,Dairy & Eggs/Milk,0.0,CARB AWARE. LOW SATURATED FAT. GO LOCAL. ...,314136,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Fat Free Milk, 1/2 gal",1.94,", 1/2 gal",($0.04/oz)
4,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314134,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients 1% Low Fat Milk, 1/2 gal",1.94,", 1/2 gal",($0.04/oz)
...,...,...,...,...,...,...,...,...,...,...
895,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314125,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Reduced Fat 2% Milk, ...",3.38,", 1 gal",($0.03/oz)
896,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314130,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Whole Milk, 1 gal",3.38,", 1 gal",($0.03/oz)
897,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314135,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Whole Milk, 1/2 gal",1.94,", 1/2 gal",($0.04/oz)
898,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,690987,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics 2% Reduced Fat Milk, 1...",3.07,", 1/2 gal",($0.05/oz)


In [8]:
heb_df.groupby(['size']).agg({'id':'count'})

,id
size,
"Whole Milk, 59 oz",15
", 1 gal",135
", 1 pt",75
", 1 qt",75
", 1/2 gal",360
", 32 oz",60
", 48 oz",30
", 52 oz",75
", 64 oz",15


In [23]:
# clean size 
clean_heb_df=pd.DataFrame()
clean_heb_df=heb_df
clean_heb_df['size']=heb_df['size'].str.rsplit(pat=',').str[1]
clean_heb_df

,brand,category,coupon,features,id,image,name,price,size,uomSalePrice
0,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931316,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics Whole Milk, 1 gal",5.85,1 gal,($0.05/oz)
1,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931317,https://images.heb.com/is/image/HEBGrocery/prd...,Central Market Organics Reduced Fat 2% Milkfat...,5.85,1 gal,($0.05/oz)
2,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS. Primo Pick.,671076,https://images.heb.com/is/image/HEBGrocery/prd...,H‑E‑B Select Ingredients MooTopia Lactose Free...,4.10,1/2 gal,($0.07/oz)
3,H-E-B,Dairy & Eggs/Milk,0.0,CARB AWARE. LOW SATURATED FAT. GO LOCAL. ...,314136,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Fat Free Milk, 1/2 gal",1.94,1/2 gal,($0.04/oz)
4,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314134,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients 1% Low Fat Milk, 1/2 gal",1.94,1/2 gal,($0.04/oz)
...,...,...,...,...,...,...,...,...,...,...
895,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314125,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Reduced Fat 2% Milk, ...",3.38,1 gal,($0.03/oz)
896,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314130,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Whole Milk, 1 gal",3.38,1 gal,($0.03/oz)
897,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314135,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Whole Milk, 1/2 gal",1.94,1/2 gal,($0.04/oz)
898,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,690987,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics 2% Reduced Fat Milk, 1...",3.07,1/2 gal,($0.05/oz)


In [39]:
clean_heb_df

,brand,category,coupon,features,id,image,name,price,size,uomSalePrice
0,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931316,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics Whole Milk, 1 gal",5.85,1 gal,NaN
1,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,931317,https://images.heb.com/is/image/HEBGrocery/prd...,Central Market Organics Reduced Fat 2% Milkfat...,5.85,1 gal,NaN
2,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS. Primo Pick.,671076,https://images.heb.com/is/image/HEBGrocery/prd...,H‑E‑B Select Ingredients MooTopia Lactose Free...,4.10,1/2 gal,NaN
3,H-E-B,Dairy & Eggs/Milk,0.0,CARB AWARE. LOW SATURATED FAT. GO LOCAL. ...,314136,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Fat Free Milk, 1/2 gal",1.94,1/2 gal,NaN
4,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314134,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients 1% Low Fat Milk, 1/2 gal",1.94,1/2 gal,NaN
...,...,...,...,...,...,...,...,...,...,...
895,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314125,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Reduced Fat 2% Milk, ...",3.38,1 gal,NaN
896,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314130,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Whole Milk, 1 gal",3.38,1 gal,NaN
897,H-E-B,Dairy & Eggs/Milk,0.0,GO LOCAL. SELECT INGREDIENTS.,314135,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Whole Milk, 1/2 gal",1.94,1/2 gal,NaN
898,Central Market,Dairy & Eggs/Milk,0.0,ORGANIC. GO LOCAL.,690987,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics 2% Reduced Fat Milk, 1...",3.07,1/2 gal,NaN


In [52]:
# unit of measure sales price - per oz float 
clean_heb_df['uomSalePrice']=heb_df['uomSalePrice'].str.strip('($/oz/fl )')
clean_heb_df['uomSalePrice']=clean_heb_df['uomSalePrice'].astype(float)

In [64]:
# convert features to list
clean_heb_df['features']=heb_df['features'].str.split(pat='.').str[:-1]

In [66]:
clean_heb_df['features'][3]

['  CARB AWARE', '  LOW SATURATED FAT', '  GO LOCAL', '  SELECT INGREDIENTS']

In [67]:
clean_heb_df

,brand,category,coupon,features,id,image,name,price,size,uomSalePrice
0,Central Market,Dairy & Eggs/Milk,0.0,"[ ORGANIC, GO LOCAL]",931316,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics Whole Milk, 1 gal",5.85,1 gal,0.05
1,Central Market,Dairy & Eggs/Milk,0.0,"[ ORGANIC, GO LOCAL]",931317,https://images.heb.com/is/image/HEBGrocery/prd...,Central Market Organics Reduced Fat 2% Milkfat...,5.85,1 gal,0.05
2,H-E-B,Dairy & Eggs/Milk,0.0,"[ GO LOCAL, SELECT INGREDIENTS, Primo Pick]",671076,https://images.heb.com/is/image/HEBGrocery/prd...,H‑E‑B Select Ingredients MooTopia Lactose Free...,4.10,1/2 gal,0.07
3,H-E-B,Dairy & Eggs/Milk,0.0,"[ CARB AWARE, LOW SATURATED FAT, GO LOCAL...",314136,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Fat Free Milk, 1/2 gal",1.94,1/2 gal,0.04
4,H-E-B,Dairy & Eggs/Milk,0.0,"[ GO LOCAL, SELECT INGREDIENTS]",314134,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients 1% Low Fat Milk, 1/2 gal",1.94,1/2 gal,0.04
...,...,...,...,...,...,...,...,...,...,...
895,H-E-B,Dairy & Eggs/Milk,0.0,"[ GO LOCAL, SELECT INGREDIENTS]",314125,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Reduced Fat 2% Milk, ...",3.38,1 gal,0.03
896,H-E-B,Dairy & Eggs/Milk,0.0,"[ GO LOCAL, SELECT INGREDIENTS]",314130,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Whole Milk, 1 gal",3.38,1 gal,0.03
897,H-E-B,Dairy & Eggs/Milk,0.0,"[ GO LOCAL, SELECT INGREDIENTS]",314135,https://images.heb.com/is/image/HEBGrocery/prd...,"H‑E‑B Select Ingredients Whole Milk, 1/2 gal",1.94,1/2 gal,0.04
898,Central Market,Dairy & Eggs/Milk,0.0,"[ ORGANIC, GO LOCAL]",690987,https://images.heb.com/is/image/HEBGrocery/prd...,"Central Market Organics 2% Reduced Fat Milk, 1...",3.07,1/2 gal,0.05


In [77]:
milk_names=clean_heb_df.groupby(['name']).agg({'id':'count'})
milk_names

,id
name,
"Blue Diamond Almond Breeze Unsweetened Original Almond Milk, 32 oz",15
"Blue Diamond Almond Breeze Unsweetened Vanilla Non-dairy Beverage, 32 oz",15
"Central Market Organics 2% Reduced Fat Milk, 1 qt",15
"Central Market Organics 2% Reduced Fat Milk, 1/2 gal",15
"Central Market Organics Fat Free Milk, 1 gal",15
"Central Market Organics Low Fat 1% Milk, 1 gal",15
"Central Market Organics Reduced Fat 2% Milkfat Milk, 1 gal",15
"Central Market Organics Whole Milk, 1 gal",15
"Central Market Organics Whole Milk, 1/2 gal",15


In [82]:
clean_heb_df.loc[clean_heb_df['id']==629469]

,brand,category,coupon,features,id,image,name,price,size,uomSalePrice
